<a href="https://colab.research.google.com/github/yuki-shi/pokedex-flask/blob/main/serebii_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports !!! 🐊

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
from collections import OrderedDict

## Webscraping !!! 🐈

In [2]:
url = 'https://serebii.net/pokemon/gen1pokemon.shtml'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

Para a extração da tabela, começamos selecionando apenas os headers.

A estrutura HTML da página é um pouco bagunçada, mas, por sorte, todos os elementos do header estão com a classe *fooevo*.

Após isso, vamos criar uma lista com o innerText das tags scrapeadas, além de limpar cada elemento de seus caractéres de separação.

In [3]:
header = []
h = soup.find_all('td', class_='fooevo')

for i in h:
  header.append(i.text)

header = [x.strip('\r\n\t') for x in header] # visto que alguns possuíam caractéres de escape
header.remove('Base Stats') # a tabela é pivotada, vamos retirar o header sobrejascente
print(header)

['No.', 'Pic', 'Name', 'Type', 'Abilities', 'HP', 'Att', 'Def', 'S.Att', 'S.Def', 'Spd']


A partir do header que conseguimos, criamos um *dicionário ordenado* usando-os como keys.

Aqui é importante usarmos tal estrutura imutável para tornar os próximos passos mais simples, mantendo-se o layout da tabela original.


Para cada key, criamos uma lista como valor.

In [4]:
poke_dict = OrderedDict.fromkeys(header)

for i in poke_dict.keys(): # talvez seja inútil, visto que usaremos .setdefault em seguida
  poke_dict[i] = []

print(poke_dict)

OrderedDict([('No.', []), ('Pic', []), ('Name', []), ('Type', []), ('Abilities', []), ('HP', []), ('Att', []), ('Def', []), ('S.Att', []), ('S.Def', []), ('Spd', [])])


Selecionamos todos os tags *tr* e alocamo-os na variável *tbody*, que sofrerá um slice a fim de retirar o header que já extraímos.

Em seguida, procuramos os *td* de cada linha iterativamente e os anexamos ao nosso dicionário de listas.

In [5]:
tbody = soup.find_all('tr')
tbody = tbody[2:]

In [6]:
for index, tr in enumerate(tbody):
  if index % 2 == 0:  # pela extração clunky, pegamos apenas os indexes pares >_>
    for i, key in enumerate(poke_dict.keys()):
      poke_dict.setdefault(key, []).append(tr.find_all('td', class_='fooinfo')[i].text.strip('\r\n\t'))

Conseguimos um dataframe inicial!

Percebemos que as colunas *Pic* e *Type* estão vazias, visto que eram compostas por imagens na tabela original.

In [7]:
df = pd.DataFrame(poke_dict)
df.head()

,No.,Pic,Name,Type,Abilities,HP,Att,Def,S.Att,S.Def,Spd
0,#001,,Bulbasaur,,Overgrow Chlorophyll,45,49,49,65,65,45
1,#002,,Ivysaur,,Overgrow Chlorophyll,60,62,63,80,80,60
2,#003,,Venusaur,,Overgrow Chlorophyll,80,82,83,100,100,80
3,#004,,Charmander,,Blaze Solar Power,39,52,43,60,50,65
4,#005,,Charmeleon,,Blaze Solar Power,58,64,58,80,65,80


## API !!! 🐿

A fim de complementar a tabela screapeada, podemos usar o uma API chamada PokéAPI para, além de ter dados mais específicos de cada Pokémon, preencher a coluna "*Types*".

No entanto, uma etapa inicial de limpeza dos nomes deve ocorrer.

### Limpeza inicial

Vemos que a tabela possui 151 linhas, o que coincide com a quantidade de Pokémons da primeira geração.

Nenhuma célula está vazia, nem mesmo *Pic* e *Type*, que estão preenchidas por strings vazias. Não iremos nos preocupar com elas por ora.


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   No.        151 non-null    object
 1   Pic        151 non-null    object
 2   Name       151 non-null    object
 3   Type       151 non-null    object
 4   Abilities  151 non-null    object
 5   HP         151 non-null    object
 6   Att        151 non-null    object
 7   Def        151 non-null    object
 8   S.Att      151 non-null    object
 9   S.Def      151 non-null    object
 10  Spd        151 non-null    object
dtypes: object(11)
memory usage: 13.1+ KB


Para a chamada da API, usaremos os nomes dos Pokémons nos endpoints.

Logo, alguns terão de ter seus nomes tratados para remover espaços e caractéres especiais.

O Pokémon *Nidoran* possui variação para macho e fêmea e, algumas vezes, como no dataframe que estamos trabalhando, representam tal diferença a nível de nome com símbolos.

Vamos os substituir por *-m* e *-f* respectivamente, a fim de se adequar ao padrão usado na API.

In [9]:
df[df['Name'].str.contains('Nidoran')]

,No.,Pic,Name,Type,Abilities,HP,Att,Def,S.Att,S.Def,Spd
28,#029,,Nidoran♀,,Poison Point Rivalry Hustle,55,47,52,40,40,41
31,#032,,Nidoran♂,,Poison Point Rivalry Hustle,46,57,40,40,40,50


In [10]:
df.loc[df['Name'].str.contains('Nidoran♂'), 'Name'] = 'Nidoran-m'
df.loc[df['Name'].str.contains('Nidoran♀'), 'Name'] = 'Nidoran-f'
df[df['Name'].str.contains('Nidoran')]

,No.,Pic,Name,Type,Abilities,HP,Att,Def,S.Att,S.Def,Spd
28,#029,,Nidoran-f,,Poison Point Rivalry Hustle,55,47,52,40,40,41
31,#032,,Nidoran-m,,Poison Point Rivalry Hustle,46,57,40,40,40,50


Usaremos, em seguida, um RegEx para checar se algum Pokémon possui espaço, apóstrofe ou ponto.

Seguimos adaptando tais ocorrências ao padrão da API.

In [11]:
df[df['Name'].str.contains(r"[ '.]", regex=True)]

,No.,Pic,Name,Type,Abilities,HP,Att,Def,S.Att,S.Def,Spd
82,#083,,Farfetch'd,,Keen Eye Inner Focus Defiant,52,90,55,58,62,60
121,#122,,Mr. Mime,,Soundproof Filter Technician,40,45,65,100,120,90


In [12]:
df['Name'] = df['Name'].str.replace("'", '', regex=False) # farfetch'd
df['Name'] = df['Name'].str.replace('.', '-', regex=False) # mr.mime
df['Name'] = df['Name'].str.replace(' ', '', regex=False)

In [13]:
df[df['Name'].str.contains('Mime')]

,No.,Pic,Name,Type,Abilities,HP,Att,Def,S.Att,S.Def,Spd
121,#122,,Mr-Mime,,Soundproof Filter Technician,40,45,65,100,120,90


### Call

Colocando todos os nomes em uma lista e convertendo-os para letras minúsculas, chamamos a API iterativamente.

Existe uma biblioteca *wrapper* para Python chamada [PokéBase](https://github.com/PokeAPI/pokebase) que realizaria tal tarefa de forma mais fácil, mas menos diertida.

In [14]:
nomes = df.loc[:, 'Name'].str.lower()
len(nomes)

151

In [15]:
def get_data(nomes, df):

  for index, nome in enumerate(nomes):

    r = requests.get(f'https://pokeapi.co/api/v2/pokemon/{nome}')
    json = r.json() if r and r.status_code == 200 else None

    try:
      # Tipos
      for i, j in enumerate(json['types']):

        if len(json['types']) == 1:
          df.loc[index, 'Type 1'] = j['type']['name']

        else:
          df.loc[index, f'Type {i+1}'] = j['type']['name']

      # Height
      df.loc[index, 'Height'] = json['height']
      
      # Weight
      df.loc[index, 'Weight'] = json['weight']

      # Abilities
      for i, j in enumerate(json['abilities']):
        df.loc[index, f'Ability {i+1}'] = j['ability']['name']
        df.loc[index, 'Hidden Ability'] = j['ability']['name'] if j['is_hidden'] else np.nan

    except:
      raise Exception(f'O pokemon {nome} deu ruim!')

  df.drop('Ability 3', axis=1, inplace=True) # temos uma coluna a mais de ability por conta do enumerate também contar a hidden   

  return df

In [16]:
df_pkm = get_data(nomes, df)
df_pkm.head(10)

,No.,Pic,Name,Type,Abilities,HP,Att,Def,S.Att,S.Def,Spd,Type 1,Type 2,Height,Weight,Ability 1,Hidden Ability,Ability 2
0,#001,,Bulbasaur,,Overgrow Chlorophyll,45,49,49,65,65,45,grass,poison,7.0,69.0,overgrow,chlorophyll,chlorophyll
1,#002,,Ivysaur,,Overgrow Chlorophyll,60,62,63,80,80,60,grass,poison,10.0,130.0,overgrow,chlorophyll,chlorophyll
2,#003,,Venusaur,,Overgrow Chlorophyll,80,82,83,100,100,80,grass,poison,20.0,1000.0,overgrow,chlorophyll,chlorophyll
3,#004,,Charmander,,Blaze Solar Power,39,52,43,60,50,65,fire,NaN,6.0,85.0,blaze,solar-power,solar-power
4,#005,,Charmeleon,,Blaze Solar Power,58,64,58,80,65,80,fire,NaN,11.0,190.0,blaze,solar-power,solar-power
5,#006,,Charizard,,Blaze Solar Power,78,84,78,109,85,100,fire,flying,17.0,905.0,blaze,solar-power,solar-power
6,#007,,Squirtle,,Torrent Rain Dish,44,48,65,50,64,43,water,NaN,5.0,90.0,torrent,rain-dish,rain-dish
7,#008,,Wartortle,,Torrent Rain Dish,59,63,80,65,80,58,water,NaN,10.0,225.0,torrent,rain-dish,rain-dish
8,#009,,Blastoise,,Torrent Rain Dish,79,83,100,85,105,78,water,NaN,16.0,855.0,torrent,rain-dish,rain-dish
9,#010,,Caterpie,,Shield Dust Run Away,45,30,35,20,20,45,bug,NaN,3.0,29.0,shield-dust,run-away,run-away


### Mais limpeza e alguns toques finais

Vamos agora remover colunas redundantes:

In [17]:
df_pkm.drop(['Type', 'Pic', 'Abilities'], axis=1, inplace=True)
df_pkm.columns

Index(['No.', 'Name', 'HP', 'Att', 'Def', 'S.Att', 'S.Def', 'Spd', 'Type 1',
       'Type 2', 'Height', 'Weight', 'Ability 1', 'Hidden Ability',
       'Ability 2'],
      dtype='object')

Sabe-se que os Pokémons lendários são usualmente alocados aos indexes finais, vamos checar:

In [18]:
df_pkm.tail(10)

,No.,Name,HP,Att,Def,S.Att,S.Def,Spd,Type 1,Type 2,Height,Weight,Ability 1,Hidden Ability,Ability 2
141,#142,Aerodactyl,80,105,65,60,75,130,rock,flying,18.0,590.0,rock-head,unnerve,pressure
142,#143,Snorlax,160,110,65,65,110,30,normal,NaN,21.0,4600.0,immunity,gluttony,thick-fat
143,#144,Articuno,90,85,100,95,125,85,ice,flying,17.0,554.0,pressure,snow-cloak,snow-cloak
144,#145,Zapdos,90,90,85,125,90,100,electric,flying,16.0,526.0,pressure,static,static
145,#146,Moltres,90,100,90,125,85,90,fire,flying,20.0,600.0,pressure,flame-body,flame-body
146,#147,Dratini,41,64,45,50,50,50,dragon,NaN,18.0,33.0,shed-skin,marvel-scale,marvel-scale
147,#148,Dragonair,61,84,65,70,70,70,dragon,NaN,40.0,165.0,shed-skin,marvel-scale,marvel-scale
148,#149,Dragonite,91,134,95,100,100,80,dragon,flying,22.0,2100.0,inner-focus,multiscale,multiscale
149,#150,Mewtwo,106,110,90,154,90,130,psychic,NaN,20.0,1220.0,pressure,unnerve,unnerve
150,#151,Mew,100,100,100,100,100,100,psychic,NaN,4.0,40.0,synchronize,NaN,NaN


Encontramos os lendários nos números #144 ao final, no entanto, #147 a #149 são Pokémons que não entram em tal categoria.

Criamos, então, uma coluna nova levando em conta tal consideração:

In [19]:
#144 ao 151 menos 147~149
df['Legendary'] = np.where((df['No.']=='#144') | 
                           (df['No.']=='#145') | 
                           (df['No.']=='#146') | 
                           (df['No.']=='#150') | 
                           (df['No.']=='#151'), 'Yes', 'No') # poderíamos deixar 0 e 1 para facilitar a EDA
df.tail(10)

,No.,Name,HP,Att,Def,S.Att,S.Def,Spd,Type 1,Type 2,Height,Weight,Ability 1,Hidden Ability,Ability 2,Legendary
141,#142,Aerodactyl,80,105,65,60,75,130,rock,flying,18.0,590.0,rock-head,unnerve,pressure,No
142,#143,Snorlax,160,110,65,65,110,30,normal,NaN,21.0,4600.0,immunity,gluttony,thick-fat,No
143,#144,Articuno,90,85,100,95,125,85,ice,flying,17.0,554.0,pressure,snow-cloak,snow-cloak,Yes
144,#145,Zapdos,90,90,85,125,90,100,electric,flying,16.0,526.0,pressure,static,static,Yes
145,#146,Moltres,90,100,90,125,85,90,fire,flying,20.0,600.0,pressure,flame-body,flame-body,Yes
146,#147,Dratini,41,64,45,50,50,50,dragon,NaN,18.0,33.0,shed-skin,marvel-scale,marvel-scale,No
147,#148,Dragonair,61,84,65,70,70,70,dragon,NaN,40.0,165.0,shed-skin,marvel-scale,marvel-scale,No
148,#149,Dragonite,91,134,95,100,100,80,dragon,flying,22.0,2100.0,inner-focus,multiscale,multiscale,No
149,#150,Mewtwo,106,110,90,154,90,130,psychic,NaN,20.0,1220.0,pressure,unnerve,unnerve,Yes
150,#151,Mew,100,100,100,100,100,100,psychic,NaN,4.0,40.0,synchronize,NaN,NaN,Yes


Lembrando que os dados scrapeados estão com tipo *object*, vamos convertê-los para *int*:

In [20]:
attrs = ['HP', 'Att', 'S.Att', 'S.Def', 'Spd']
df[attrs] = df[attrs].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   No.             151 non-null    object 
 1   Name            151 non-null    object 
 2   HP              151 non-null    int64  
 3   Att             151 non-null    int64  
 4   Def             151 non-null    object 
 5   S.Att           151 non-null    int64  
 6   S.Def           151 non-null    int64  
 7   Spd             151 non-null    int64  
 8   Type 1          151 non-null    object 
 9   Type 2          67 non-null     object 
 10  Height          151 non-null    float64
 11  Weight          151 non-null    float64
 12  Ability 1       151 non-null    object 
 13  Hidden Ability  145 non-null    object 
 14  Ability 2       145 non-null    object 
 15  Legendary       151 non-null    object 
dtypes: float64(2), int64(5), object(9)
memory usage: 19.0+ KB


Possuímos agora algumas variáveis nulas, o que já era esperado, visto que nem todos Pokémons possuem *Type 2*, *Ability 2* ou *Hidden Ability*:

In [21]:
df[df['Hidden Ability'].isnull()]

,No.,Name,HP,Att,Def,S.Att,S.Def,Spd,Type 1,Type 2,Height,Weight,Ability 1,Hidden Ability,Ability 2,Legendary
10,#011,Metapod,50,20,55,25,25,30,bug,NaN,7.0,99.0,shed-skin,NaN,NaN,No
13,#014,Kakuna,45,25,50,25,25,35,bug,poison,6.0,100.0,shed-skin,NaN,NaN,No
91,#092,Gastly,30,35,30,100,35,80,ghost,poison,13.0,1.0,levitate,NaN,NaN,No
92,#093,Haunter,45,50,45,115,55,95,ghost,poison,16.0,1.0,levitate,NaN,NaN,No
93,#094,Gengar,60,65,60,130,75,110,ghost,poison,15.0,405.0,cursed-body,NaN,NaN,No
150,#151,Mew,100,100,100,100,100,100,psychic,NaN,4.0,40.0,synchronize,NaN,NaN,Yes


Por fim, reordenamos as colunas, por capricho talvez, organizando as variáveis categóricas ao início e fim.

In [22]:
df_pkm = df_pkm[['No.', 'Name', 'Type 1', 'Type 2', 'HP', 'Def', 'S.Att', 'S.Def', 'Spd', 'Height', 'Weight', 'Ability 1', 'Ability 2', 'Hidden Ability', 'Legendary']]
df_pkm.head()

,No.,Name,Type 1,Type 2,HP,Def,S.Att,S.Def,Spd,Height,Weight,Ability 1,Ability 2,Hidden Ability,Legendary
0,#001,Bulbasaur,grass,poison,45,49,65,65,45,7.0,69.0,overgrow,chlorophyll,chlorophyll,No
1,#002,Ivysaur,grass,poison,60,63,80,80,60,10.0,130.0,overgrow,chlorophyll,chlorophyll,No
2,#003,Venusaur,grass,poison,80,83,100,100,80,20.0,1000.0,overgrow,chlorophyll,chlorophyll,No
3,#004,Charmander,fire,NaN,39,43,60,50,65,6.0,85.0,blaze,solar-power,solar-power,No
4,#005,Charmeleon,fire,NaN,58,58,80,65,80,11.0,190.0,blaze,solar-power,solar-power,No


In [23]:
df.to_csv('pokmono.csv', index=False)